In [ ]:
import os
from tqdm import tqdm
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from open3d.web_visualizer import draw


In [2]:
from quickstart.data import point_cloud_data
from quickstart.code.preprocess.data_gen import crop_pc_z, get_density


# Load point cloud data

In [ ]:
pcd_path = "quickstart/data/aihub/S_01_0001_1_I.ply"
pcd = point_cloud_data.read_point_cloud(pcd_path)
pcd  # PointCloud with 1941471 points.


# Oriented Object Bounding Box

In [ ]:
bbox = o3d.geometry.OrientedBoundingBox.create_from_points(pcd.points)
bbox  # OrientedBoundingBox


# Crop


In [5]:
# 바운딩 박스 내에 있는 포인트들만 크롭
cropped_pcd = pcd.crop(bbox)


In [ ]:
# 크롭된 포인트 클라우드 시각화
draw([cropped_pcd])


# Slicing

## Slicing intervals

In [ ]:
min_z = bbox.get_min_bound()[-1]
max_z = bbox.get_max_bound()[-1]
slice_intervals = np.linspace(min_z, max_z, 10)[1:-1]
slice_intervals


In [8]:
center_slice_points = []
for cz in slice_intervals:
    cx, cy, _ = bbox.center
    center_slice_points.append([cx, cy, cz])
centers_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(center_slice_points))


In [ ]:
centers_pcd.colors = o3d.utility.Vector3dVector(
    np.array([[1, 0, 0] for _ in range(len(center_slice_points))])
)
draw([cropped_pcd, centers_pcd])


## Bounds

In [10]:
bounds = (*bbox.get_min_bound()[:2], *bbox.get_max_bound()[:2])


## Slice

In [ ]:
width, height = 1462, 1462
density_slices = []
for slice_i, (slice_minz, slice_maxz) in enumerate(
    tqdm(list(zip(slice_intervals[:-1], slice_intervals[1:])))
):
    sliced_pcd = crop_pc_z(cropped_pcd, slice_minz, slice_maxz)
    density_slice = get_density(sliced_pcd, bounds, width, height)
    density_slices.append(density_slice)


In [14]:
floor_name = "S_01_0001_1_I"
if not os.path.exists(f"quickstart/data/density/{floor_name}"):
    os.makedirs(f"quickstart/data/density/{floor_name}")
    for i in range(7):
        npy_path = f"quickstart/data/density/{floor_name}/density_{i:02d}.npy"
        np.save(npy_path, density_slices[i])


In [ ]:
num_plot = len(density_slices)  # 7
fig, axs = plt.subplots(3, 3, figsize=(15, 15))
for i in range(9):  # 0..6
    ax = axs[i // 3, i % 3]
    if i >= num_plot:
        ax.set_visible(False)
        ax.axis("off")
    else:
        ax.imshow(density_slices[i])
plt.show()
